# Brick algorithm

# Mathis

In [6]:
# Line search

import pandas as pd
data = pd.read_csv('Iris_modified.csv', index_col = 0)

In [7]:
# These are the data I use to compute the functional we want to minimize (i.e f(Beta))
data.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,-1
1,4.9,3.0,1.4,0.2,-1
2,4.7,3.2,1.3,0.2,-1
3,4.6,3.1,1.5,0.2,-1
4,5.0,3.6,1.4,0.2,-1


In [68]:
# Import modules
import numpy as np

In [69]:
# Here I create the data matrix X, vector of response Y and vector of coefficients beta
Y = data.Species.values
X = data[data.columns.difference(['Species'])].as_matrix()
beta = np.random.normal(size=4) # Here I simulate random coefficients from a normal 0,1 

$f(\beta) = L(\beta) + \lambda ||\beta||_1$ where $L(\beta) = \sum\limits_{i=1}^nlog(1+e^{-y_i\beta^Tx_i})$

In [70]:
def loss_function(X,Y,beta):
    y_beta_x = np.multiply(Y,X.dot(beta))
    return sum(np.log(1+np.exp(-y_beta_x)))

In [71]:
def penalty(beta,regularization):
    return regularization*np.linalg.norm(beta, ord=1)

In [72]:
# This is f(beta)
def functional(X,Y,beta,regularization):
    return loss_function(X,Y,beta)+penalty(beta,regularization)

__Algorithm 3:__ Line search procedure:

1. If $\alpha = 1$ yields sufficient relative decrease in the objective, return $\alpha=1$

2. Find $\alpha_{init} = argmin_{\delta<\alpha<1} \text{  } f(\beta+\alpha\Delta\beta)$, $\delta >0$

3. Armijo rule: Let $\alpha$ be the largest element of the sequence $\{\alpha_{init}b^j\}_{j=0,1,...}$ satisfying

<h1><center>
$f(\beta+\alpha\Delta\beta) \leq f(\beta)+\alpha\sigma D$
</center></h1>

where $0<b<1$, $0<\sigma<1$, $0\leq\gamma<1$, and

<h1><center>
$D = \nabla L(\beta)^T \Delta\beta+\gamma\Delta\beta^T\tilde{H}\Delta\beta+\lambda(||\beta-\Delta\beta||_1-||\beta||_1)$
</center></h1>

return $\alpha$

What is difficult here is to compute D. In what follows, I explain how is computed $\nabla L(\beta)$ and $\tilde{H}$:

- $\nabla L(\beta) = \frac{dL(\beta)}{d\beta} = (\frac{dL(\beta)}{d\beta_1}, \frac{dL(\beta)}{d\beta_2},...) \text{  avec  } L(\beta) = \sum\limits_{i=1}^nlog(1+e^{-y_i\beta^Tx_i}) = \sum\limits_{i=1}^nlog(1+e^{-y_i(\beta_{1i}x_{1i}+\beta_{2i}x_{2i}+...)})$

So it comes that $\nabla L(\beta) = \bigg(\sum\limits_{i=1}^n\frac{-y_ix_{1i}e^{-y_i(\beta_{1i}x_{1i}+\beta_{2i}x_{2i}+...)}}{1+e^{-y_i(\beta_{1i}x_{1i}+\beta_{2i}x_{2i}+...)}}, \sum\limits_{i=1}^n\frac{-y_ix_{2i}e^{-y_i(\beta_{1i}x_{1i}+\beta_{2i}x_{2i}+...)}}{1+e^{-y_i(\beta_{1i}x_{1i}+\beta_{2i}x_{2i}+...)}},...\bigg) = \bigg(\sum\limits_{i=1}^n\frac{-y_ix_{1i}e^{-y_i\beta^Tx_i}}{1+e^{-y_i\beta^Tx_i}}, \sum\limits_{i=1}^n\frac{-y_ix_{2i}e^{-y_i\beta^Tx_i}}{1+e^{-y_i\beta^Tx_i}},...\bigg)$

In [73]:
def gradient_loss(X,Y,beta):
    gradient = []
    for j in range(len(beta)):
        y_beta_x = Y*X.dot(beta)
        y_xj = Y*X[:,j]
        gradient_betaj = sum((-y_xj*np.exp(-y_beta_x))/(1+np.exp(-y_beta_x)))
        gradient.append(gradient_betaj)
    return np.array(gradient)

- $\tilde{H}$ is such that $(\tilde{H})_{jl} =
    \begin{cases}
    (\nabla^2L(\beta))_{jl},& \text{if } \exists m:j,l \in S_m\\
    0,              & \text{otherwise}
    \end{cases}$

To compute the $\tilde{H}$ matrix, we won't suppose any partitioning from now. We suppose that all our variables are on one machine.

To do this computation, we need first to define the second derivative of the loss between two variables $j$ and $l$.

$(\nabla^2L(\beta))_{jl} = \sum\limits_{i=1}^n\frac{y_i^2x_{ji}x_{li}e^{-y_i\beta^Tx_i}(1+e^{-y_i\beta^Tx_i}) + y_ix_{li}e^{-y_i\beta^Tx_i}(-y_ix_{ji}e^{-y_i\beta^Tx_i})}{(1+e^{-y_i\beta^Tx_i})^2}$

In [74]:
def second_gradient_loss(X,Y,beta,variable_j,variable_l):
    y_beta_x = Y*X.dot(beta)
    ysq_xj_xl = Y**2*(X[:,variable_j]*X[:,variable_l])
    gradient_betajl = sum((ysq_xj_xl*np.exp(-y_beta_x)*(1+np.exp(-y_beta_x)) + Y*(X[:,variable_l])*np.exp(-y_beta_x)*(-Y*(X[:,variable_j])*np.exp(-y_beta_x)))/(1+np.exp(-y_beta_x))**2)
    return gradient_betajl

In [75]:
import itertools
def H_tilde(X,Y,beta):
    # Combination of all possible indexes
    iterator = list(itertools.product([i for i in range(len(beta))], [i for i in range(len(beta))]))
    # Initialize H_tilde matrix
    H_tilde = np.empty((len(beta),len(beta),))
    H_tilde[:] = np.nan
    for i in iterator:
        H_tilde[i[0],i[1]] = second_gradient_loss(X,Y,beta,i[0],i[1])
    return H_tilde

Therefore, one might compute D using:

In [76]:
def D(X,Y,beta,regularization):
    return gradient_loss(X,Y,beta).dot(delta_B) + gamma*delta_B.dot(H_tilde(X,Y,beta)).dot(delta_B) + regularization*(np.linalg.norm(beta-delta_B, ord=1)-np.linalg.norm(beta, ord=1))

Now, here is out line search function:

In [77]:
from scipy import optimize

def line_search(X,Y,beta,regularization, delta_B, sufficient_decrease, delta, b, sigma, gamma):
    if functional(X,Y,beta,regularization) - functional(X,Y,beta+delta_B,regularization) > sufficient_decrease:
        return 1
    else:
        result = optimize.minimize_scalar(lambda x: functional(X,Y,beta+x*delta_B,regularization), bounds=(delta,1), method='Bounded')
        alpha_in = result.x
        alpha = alpha_in
        count = 0
        while functional(X,Y,beta+alpha*delta_B,regularization) > functional(X,Y,beta,regularization) + alpha*sigma*D(X,Y,beta,regularization):
            count += 1
            alpha *= b**count
    return alpha

In [88]:
# Set
regularization = 3
delta_B = np.array([.9,1,3,4])
sufficient_decrease = 1000
delta = 0.2
b = .5
sigma = .01
gamma = 0

In [89]:
line_search(X,Y,beta,regularization, delta_B, sufficient_decrease, delta, b, sigma, gamma)

5.5512366814618351e-18